# European Soccer

## Extracting Data
dataset available at: https://www.kaggle.com/hugomathien/soccer/data

In [1]:
import csv
import sqlite3 as lite

In [4]:
conn = lite.connect('database.sqlite')
c = conn.cursor()

c.execute('SELECT `rowid`,* FROM `Match` WHERE `country_id` LIKE \'4769\' ORDER BY `rowid` ASC;')
# country_id <=> ligue_id, i.e. pas de match de coupe
descr = c.description
champs = [d[0] for d in descr]
# print(champs) # si tu veux voir sur quoi on peut tabuler

"""
la suite du code plante John :(
"""

## Ligue 1 conforama

In [6]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('ligue1_conforama.csv')

In [9]:
df.head()

,id,id.1,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,4769,4769,4769,4769,2008/2009,1,2008-08-09 00:00:00,483129,8583,9830,...,3.8,2.0,2.9,3.75,2.10,3.00,3.75,2.00,2.88,3.75
1,4770,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,...,7.5,1.6,3.3,5.50,1.57,3.60,6.00,1.50,3.50,6.50
2,4771,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,...,3.1,2.3,2.9,3.00,2.35,2.90,3.20,2.38,2.88,2.88
3,4772,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,...,4.0,2.0,2.9,3.75,2.00,3.10,3.75,1.91,3.00,4.00
4,4773,4773,4769,4769,2008/2009,1,2008-08-10 00:00:00,483133,9748,9941,...,12.0,1.3,4.0,10.00,1.36,4.25,9.00,1.30,4.33,9.00


### On mesure la probabilité empirique pour l'équipe qui accueille de V/N/D

In [10]:
df['goal_diff'] = df['home_team_goal'] - df['away_team_goal']

In [11]:
def delta_to_result(delta):
    if delta > 0:
        return 'V'
    if delta == 0:
        return 'N'
    else:
        return 'D'

In [12]:
df['home_team_result'] = df['goal_diff'].map(delta_to_result)

In [13]:
df['home_team_result'].head()

0    V
1    V
2    V
3    D
4    V
Name: home_team_result, dtype: object

In [19]:
df.groupby('home_team_result').count()

,id,id.1,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,goal_diff
home_team_result,,,,,,,,,,,,,,,,,,,,,
D,822,822,822,822,822,822,822,822,822,822,...,822,822,822,495,495,495,495,495,495,822
N,859,859,859,859,859,859,859,859,859,859,...,859,859,859,555,555,555,555,555,555,859
V,1359,1359,1359,1359,1359,1359,1359,1359,1359,1359,...,1356,1356,1356,847,847,847,847,847,847,1359


In [22]:
pV = 1359. / len(df.index)
pN = 859. / len(df.index)
pD = 822. / len(df.index)

In [24]:
# empiric probas of V/N/D for the home_team
p = np.array([pV, pN, pD])
p

array([ 0.44703947,  0.28256579,  0.27039474])